In [1]:
import pydicom
from tqdm import tqdm_notebook as tqdm
from glob import glob
import os
import shutil

RENAME_DICT = {
    'zero': 'zerodose',
    'low': 'lowdose',
    'full': 'fulldose'
}

def write_accession_number(dirpath_series):
    dcm_files = [(pydicom.dcmread(f), f) for f in glob('{}/*.dcm'.format(dirpath_series))]
    
    for dcm_file in tqdm(dcm_files, total=len(dcm_files)):
        new_acc_num = str(dcm_file[0].PatientName)
        dcm_file[0].AccessionNumber = new_acc_num
        dcm_file[0].save_as(dcm_file[1])

def find_post_contrast_series(dirpath_case):
    return sorted(
        [ser for ser in glob('{}/*'.format(dirpath_case))],
        key=lambda ser: int(ser.split('/')[-1].split('_')[0])
    )[0]

def rename_series(dirpath_series, desc_suffix, sep='_', cs=False):
    dcm_files = [(pydicom.dcmread(f), f) for f in glob('{}/*.dcm'.format(dirpath_series))]
    if cs:
        desc_suffix = '_cs'

    print('Renaming {} with "{}" suffix'.format(dirpath_series, desc_suffix))
    
    curr_desc = dcm_files[0][0].SeriesDescription
    if not curr_desc:
        curr_desc = dirpath_series.split('/')[-2]
    
    print('Current series description', curr_desc)
    
    series_desc = '{}{}{}'.format(curr_desc, sep, desc_suffix)
    
    for dcm_file in tqdm(dcm_files, total=len(dcm_files)):
        dcm_file[0].SeriesDescription = series_desc
        dcm_file[0].save_as(dcm_file[1])

def process_renaming(dirpath_study, cs=False):
    dict_snum = {}
    keys = ['zero', 'low', 'full']
    
    dirpaths_series = [d for d in glob('{}/*'.format(dirpath_study))]
    
    for series_path in dirpaths_series:
        dcm_files = [f for f in glob('{}/*.dcm'.format(series_path))]
        dcm = pydicom.dcmread(dcm_files[0])
        dict_snum[int(dcm.SeriesNumber)] = series_path
    
    dict_suffixes = {}
    for i, snum in enumerate(sorted(dict_snum.keys())):
        dict_suffixes[RENAME_DICT[keys[i]]] = dict_snum[snum]
    
    for k, v in dict_suffixes.items():
        rename_series(v, k, cs)

def find_bpath_for_case(case_num):
    dirbase = '/Users/srivathsa/projects/studies/gad'
    dirsites = ['stanford', 'hoag', 'tiantan', 'sford_siemens', 'all']
    for site_dir in dirsites:
        dirpath_site = os.path.join(dirbase, site_dir, case_num)
        if os.path.isdir(dirpath_site):
            return dirpath_site
    return None

In [ ]:
cases = [
    "101_Id_051", "101_Id_066", "Id0032", "NO108", "NO113", "NO120", "NO129", "NO130", "NO18", "NO26", "NO54",
    "NO55", "NO56", "NO6", "NO60", "NO62", "NO67", "NO71", "NO79", "Patient_0087", "Patient_0090", "Patient_0134",
    "Patient_0157", "Patient_0172", "Patient_0173", "Patient_0178", "Patient_0255", "Patient_0269", "Patient_0276",
    "Patient_0286", "Patient_0333", "Patient_0342", "Patient_0353", "Patient_0375", "Patient_0400", "Patient_0408",
    "Patient_0486", "Patient_0526", "Patient_0535", "Patient_0538", "Patient_0556", "Patient_0575", "Prisma1",
    "Prisma21", "Prisma22", "Prisma23", "Prisma3", "Prisma4", "Prisma6", "Prisma9"
]

In [ ]:
for case in cases:
    dirpath_study = find_bpath_for_case(case)
    process_renaming(dirpath_study)

In [ ]:
dirpath_dcm = '/Users/srivathsa/projects/studies/gad/image_review/rfm_cases/Prisma4/SubtleGADt1_mprage_sag_p2_isoresearch_only_105_RFM'
# dirpath_dcm = '/Users/srivathsa/projects/studies/gad/stanford/Patient_0085/14_AX_BRAVO+C'
write_accession_number(dirpath_dcm)

In [2]:
base_dir = '/home/srivathsa/subtlemr/input/sgad'
suffix = 'DNE'

dirpath_cases = [d for d in glob('{}/*'.format(base_dir))]

for dpath_case in dirpath_cases:
    ser_dirs = [s for s in glob('{}/*'.format(dpath_case)) if os.path.isdir(s)]
    
    for sdir in ser_dirs:
        rename_series(sdir, suffix, sep=' ')

Renaming /home/srivathsa/subtlemr/input/sgad/1231970/1231970_SubtleGad with "DNE" suffix
Current series description SubtleGad:  8_st1w_3d_ffe+csford_enh_vgg_3r


/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/180 [00:00<?, ?it/s]

Renaming /home/srivathsa/subtlemr/input/sgad/1755456/1755456_SubtleGad with "DNE" suffix
Current series description SubtleGad:  5_ax_3d_fspgr_mdcsford_enh_vgg_3r


  0%|          | 0/100 [00:00<?, ?it/s]

Renaming /home/srivathsa/subtlemr/input/sgad/1002071/1002071_SubtleGad with "DNE" suffix
Current series description SubtleGad:  9_st1w_3d_isotiantan_enh_vgg_3r


  0%|          | 0/150 [00:00<?, ?it/s]

### Group processed series

In [3]:
app_output = '/home/srivathsa/subtlemr/output/dicoms/sgad'

for dpath_series in glob('{}/*'.format(app_output)):
    if not os.path.isdir(dpath_series):
        continue
    
    dcm_files = sorted([fp for fp in glob('{}/*.dcm'.format(dpath_series))])
    
    dcm = pydicom.dcmread(dcm_files[0])
    patient_id = dcm.PatientID
    ser_desc = dcm.SeriesDescription.replace(' DNE', '').replace(' SRE', '').replace(' SMR', '').replace(' ', '')
    outdir = '{}/{}/{}'.format(app_output, patient_id, ser_desc)
    os.makedirs(outdir, exist_ok=True)
    
    for fpath_dcm in tqdm(dcm_files, total=len(dcm_files)):
        fname = fpath_dcm.split('/')[-1]
        dcm = pydicom.dcmread(fpath_dcm)
        dcm.SeriesDescription = ser_desc
        dcm.StudyDescription = ser_desc
        dcm.save_as('{}/{}'.format(outdir, fname))
    
    shutil.rmtree(dpath_series)

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]